##### Import and config

In [3]:
import pandas as pd
import os
from sqlalchemy import create_engine
import datetime as dt
import time

import Update_CM as uc

days_to_get = 4
plink = 'C:/PBI/Claro/data/extras/PLink/GENOPT/'
plink_file = '【NPOC】Task Table.csv'
db_user = 'Juliano Correa'

In [4]:
#uc.__init__(db_user, days_to_get)
cm_form_open = uc.get_cm()


##### Read Task Export

In [5]:
df_plink = pd.read_csv(plink + plink_file, parse_dates=True)
#df_plink_rfsh = df_plink[df_plink['Task List'] == "RF Shaping"].copy()
df_plink_all = df_plink[df_plink['Task List'].isin(['RF Shaping','Logic Parameter','Traffic Balance'])].copy()
#df_plink_all = df_plink[df_plink['Task List'].isin(['Logic Parameter','Traffic Balance'])].copy()
#df_plink_rfsh_c = df_plink_rfsh.copy()
df_plink_all["Update Time"] = pd.to_datetime(df_plink_all["Update Time"])
df_plink_all = df_plink_all[df_plink_all["Update Time"].dt.date > uc.days_to_check(days_to_get)]
df_plink_all.columns = df_plink_all.columns.astype(str).str.replace(" ", "_")
df_plink_all.columns = map(str.lower, df_plink_all.columns)
#df_plink_rfsh_test = df_plink_rfsh[df_plink_rfsh.content == '55S01BAAMA9403']
#df_plink_rfsh_test.head()
df_plink_rfsh_test = df_plink_all
#df_plink_rfsh_test = df_plink_all[df_plink_all['content'].isin(['55S01PITER4801'])]
df_plink_rfsh_test.head(2)


,content,project,start_date,due_date,creation_time,update_time,complete_time,completed_or_not,overdue,taskid,...,workflow_status,task_type,tag,note,executor,task_list,task_group,creator,priority,followers
1010,ES01PECRU06,NPOC,2024-05-03 09:00:00,2024-06-07 18:00:00,2024-03-25 20:21:11,2024-05-03 11:20:27,NaN,N,N,NPoC-25579,...,NaN,Task,C | Cell Mapping | 4G | GEN OPT,02/04 - Analisar balanceamento,Bryan Lucas Menezes Batista(bryan.lucas.batist...,Traffic Balance,GENERAL OPTIMIZATION,Juliano Corrêa(juliano.correa.myb@h-partners.com),Default,Juliano Corrêa | Bryan Lucas Menezes Batista
1011,ES01PEOLD07,NPOC,2024-05-03 09:00:00,2024-06-07 18:00:00,2024-03-25 20:21:10,2024-05-03 11:22:08,NaN,N,N,NPoC-25574,...,NaN,Task,C | Cell Mapping | 4G | GEN OPT,02/04 - Analisar balanceamento,Bryan Lucas Menezes Batista(bryan.lucas.batist...,Traffic Balance,GENERAL OPTIMIZATION,Juliano Corrêa(juliano.correa.myb@h-partners.com),Default,Juliano Corrêa | Bryan Lucas Menezes Batista


##### Main

In [9]:
for row in df_plink_rfsh_test.itertuples():
    pl_tag = row.tag
    pl_object = row.content
    if len(pl_object) == 14:
        pl_cell = pl_object
        pl_node = ''
        if pl_cell[:1] == '5':
            pl_tec = 'nr'
        else:
            pl_tec = 'lte'
    elif len(pl_object) == 10:
        pl_cell = ''
        pl_node = pl_object[-7:]
        if '4G' in pl_tag:
            pl_tec = 'lte'
        elif '5G' in pl_tag:
            pl_tec = 'nr'
    pl_taskid = row.taskid
    pl_note = row.note
    pl_note_r = pl_note.replace('%','%%')
    pl_s_date = row.start_date
    pl_completed_or_not = row.completed_or_not
    pl_complete_time = str(row.complete_time)
    pl_owner = ''
    pl_tasklist = row.task_list
    if 'RTL' in pl_tag:
        pl_owner = 'RTL'
    elif 'GEN OPT' in pl_tag:
        pl_owner = 'GEN OPT'
    _data_plan = pl_s_date[:10]
    if pl_complete_time != 'nan':
        _data_term = pl_complete_time[:10]
    else:
        _data_term = ''    
    _acao = f'{pl_note_r}'
    print(pl_tasklist)
    if pl_tasklist == 'RF Shaping':
        uc.check_cell_task(pl_cell, pl_taskid, _data_plan, _data_term, _acao, pl_tasklist, pl_owner, pl_note_r, pl_tec, cm_form_open, db_user)
        #print(pl_node, pl_cell,  pl_note, pl_owner,  pl_taskid, pl_s_date[:10], pl_tec, pl_completed_or_not, pl_complete_time, pl_tasklist)
    if pl_tasklist == 'Traffic Balance':
        uc.check_node_task(pl_node, pl_taskid, _data_plan, _data_term, _acao, pl_tasklist, pl_owner, pl_note_r, pl_tec, cm_form_open, db_user)
    if pl_tasklist == 'Logic Parameter':
        uc.check_node_task(pl_node, pl_taskid, _data_plan, _data_term, _acao, pl_tasklist, pl_owner, pl_note_r, pl_tec, cm_form_open, db_user)    
    #
    
    print('--------------') 
    

Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1078, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 297, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "c:\Users\JC_A\anaconda3\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 1949, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "c:\Users\JC_A\anaconda3\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 1984, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 

In [5]:
print('fim')

fim
